In [26]:
import numpy as np

Определяем размерность задачи.

In [27]:
L = 10
h = 1/L

In [28]:
print (h)

0.1


In [29]:
l_arr = [i for i in range(0, L+1)]
x_arr = [0 + l*h for l in l_arr]

In [30]:
print (l_arr)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [31]:
print(x_arr)

[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0]


Определяем функции из условия основной задачи

In [32]:
k_initial = lambda x: np.power(np.e, -np.sin(2*np.pi*x))
#k_initial = lambda x: np.cos(2* np.pi*x)**2 + 1
q_initial = lambda x: 1
#q_initial = lambda x: np.sin(2*np.pi*x) + 1.5
f_initial = lambda x: np.cos(2*np.pi*x)
#f_initial = lambda x: np.sin(2*np.pi*x)

Определяем функции из условия модельной задачи.

In [33]:
kmod = k_initial(0)
qmod = q_initial(0)
fmod = lambda x: np.cos(2* np.pi * x) + 1
#fmod = lambda x: np.sin(2*np.pi*x)

Определяем аналитическое решение нашей задачи.

In [34]:
u_anal = lambda l: 1 + 1/(4* np.pi**2 +1)*f_initial(x_arr[l])

Следующие функции верны для обоих систем:

In [36]:
al = lambda l: kmod
bl = lambda l: -2*kmod - qmod*h**2
cl = lambda l: kmod

## Определяем функцию, реализующую метод прогонки.

In [37]:
def progonka_progonochka(al, bl, cl, dl, a0, b0, d0, bL, cL, dL, L):
    alpha = [None]*(L+1)
    beta = [None]*(L+1)
    u = [None]*(L+1)
    alpha[0] = - a0/b0
    beta[0] = d0/b0
    #прямая прогонка
    for l in range(1,L):
        alpha[l] = -al(l)/(bl(l) + cl(l)*alpha[l-1] )
        beta[l] = (dl(l) - cl(l)*beta[l-1])/(bl(l) + cl(l)*alpha[l-1])
    u[L] = (dL - cL*beta[L-1])/(bL + cL*alpha[L-1])    
    #обратная прогонка
    for l in range(L-1,-1,-1 ):
        u[l] = alpha[l]*u[l+1] + beta[l]
    return u

Решение системы ищем в виде $ u^{(h)} = v^{(h)} + u_0 w^{(h)} $, где  $v^{(h)}  $ и $w^{(h)}$ сеточные функции, а $ u_0 = const$

### Решаем первую систему

In [38]:
dl_1 = lambda l: -h**2 * fmod(x_arr[l])
a0_1 = 0
b0_1 = 1
d0_1= 0
bL_1 = 1
cL_1 = 0
dL_1 = 0
v_mod = progonka_progonochka(al, bl, cl, dl_1, a0_1, b0_1, d0_1, bL_1, cL_1, dL_1, L)

### Решаем вторую систему

In [39]:
dl_2 = lambda l: 0
a0_2 = 0
b0_2 = 1
d0_2 = 1
bL_2 = 1
cL_2 = 0
dL_2 = 1
w_mod = progonka_progonochka(al, bl, cl, dl_2, a0_2, b0_2, d0_2, bL_2, cL_2, dL_2, L)

### Находим $u_0$

In [40]:
u_0_mod = (kmod*(v_mod[1] + v_mod[L-1]) + fmod(0)*h**2)/(2*kmod + qmod*h**2 - kmod*(w_mod[1] + w_mod[L - 1]))

### Записываем решение

In [41]:
def solution(v, w, u_0, disp = True):
    u = np.array(v) + np.array(w)*u_0
    if disp:
        print(u)
    return u
    

In [42]:
u_mod = solution(v_mod, w_mod, u_0_mod, disp = True)

[ 1.02551242  1.02063998  1.00788377  0.99211623  0.97936002  0.97448758
  0.97936002  0.99211623  1.00788377  1.02063998  1.02551242]


## Записываем аналитическое решение

In [43]:
print ([u_anal(l) for l in l_arr ])

[1.0247045230318577, 1.0199863789707002, 1.0076341174547714, 0.99236588254522873, 0.9800136210292999, 0.97529547696814234, 0.9800136210292999, 0.99236588254522873, 1.0076341174547714, 1.0199863789707002, 1.0247045230318577]


## Считаем ошибку на модельной задаче.

In [44]:
print (max([u_anal(l) - u_mod[l] for l in l_arr]))

0.000807893129174


### Определяем функции из условия задачи

In [45]:
q = lambda l: q_initial(x_arr[0])
f = lambda l: f_initial(x_arr[l])

In [46]:
#def k(l=0,r = True):
 #   if r :
  #      return np.power(np.e, - np.sin(2*np.pi*(x_arr[l] + h/2)))
   # else:
    #    return np.power(np.e, - np.sin(2*np.pi*(x_arr[l] - h/2)))
    

In [47]:
def k(l = 0, r = True):
    if r:
        return k_initial(x_arr[l] + h/2)
    else:
        return k_initial(x_arr[l] - h/2)

Вводим обозначения:

In [48]:
a = lambda l: k(l)
b = lambda l: -(k(l) + k(l, r = False) + q(l)*h**2)
c = lambda l: k(l, r = False)

Снова ищем решение по методу циклической прогонки, т.е в виде суммы решений двух систем с каким-то коэффициентом. Заметим, что решение этих двух систем отличается от решения этих же систем в модельной задаче только функциями a,b,c, и d.

### Решаем первую систему

In [49]:
d_1 = lambda l: -f(l)*h**2
v = progonka_progonochka(a, b, c, d_1, a0_1, b0_1, d0_1, bL_1, cL_1, dL_1, L)

### Решаем вторую систему

In [50]:
d_2 = lambda x: 0
w = progonka_progonochka(a, b, c, d_2, a0_2, b0_2, d0_2, bL_2, cL_2, dL_2, L)

### Находим $u_0$

In [51]:
u_0 = (k(0,1)*v[1]+ k(L,False)*v[L-1] + f(0)*h**2)/(k(0,1) + k(L, False) + q(0)*h**2 - k(0,1)*w[1] - k(L,False)*w[L-1])

In [52]:
#u_0 = (a(0)*v[1]+ c(L)*v[L-1] + f(0)*h**2)/(a(0) + c(L) + q(0)*h**2 - a(0)*w[1] - c(L)*w[L-1])

### Записываем решение

In [53]:
u = solution(v,w,u_0)

[ 0.02179366  0.02467852  0.01182096 -0.01182096 -0.02467852 -0.02179366
 -0.01305704 -0.00421363  0.00421363  0.01305704  0.02179366]
